# SAE Labyrinthes

___

Léo MATHIEU | Shane GOBIN

In [809]:
import random

## 3 Implémentation

Nous allons définir la classe `Maze` à l’aide des attributs :

- `height`, le nombre de **lignes** (`int`) de la grille du labyrinthe (autrement dit, la hauteur, en nombre de cellules),
- `width`, le nombre de **colonnes** (`int`) de la grille du labyrinthe (autrement dit, la hauteur, en nombre de cellules),
- `neighbors` : un dictionnaire (`dict`) qui associe à chaque cellule, un `set` contenant ses **voisines** (c’est-à-dire les cellules qu’on peut atteindre en un déplacement1, sans être bloqué par un mur).

Voici donc la définition sommaire de la classe Maze, pour laquelle nous vous fournissons, un constructeur par défaut, une méthode d’affichage (en ASCII), et une méthode qui résume les infos du labyrinthe :

In [810]:
class Maze:
    """
    Classe Labyrinthe
    Représentation sous forme de graphe non-orienté
    dont chaque sommet est une cellule (un tuple (l,c))
    et dont la structure est représentée par un dictionnaire
      - clés : sommets
      - valeurs : ensemble des sommets voisins accessibles
    """
    def __init__(self, height, width, empty = False):
        """
        Constructeur d'un labyrinthe de height cellules de haut 
        et de width cellules de large 
        Les voisinages sont initialisés à des ensembles vides
        Le booléen empty permet d'indiquer si le labyrinthe est créé avec (True) ou sans (False)
        que les cellules aient de voisin
        Remarque : dans le labyrinthe créé, chaque cellule est complètement emmurée
        """
        self.height    = height
        self.width     = width
        self.neighbors = {(i,j): set() for i in range(height) for j in range (width)}
        if empty:
            for i in range(height):
                for j in range(width):
                    if i > 0:
                        self.neighbors[(i,j)].add((i-1,j))
                        self.neighbors[(i-1,j)].add((i,j))
                    if j > 0:
                        self.neighbors[(i,j)].add((i,j-1))
                        self.neighbors[(i,j-1)].add((i,j))

    def info(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Affichage des attributs d'un objet 'Maze' (fonction utile pour deboguer)
        Retour:
            chaîne (string): description textuelle des attributs de l'objet
        """
        txt = "**Informations sur le labyrinthe**\n"
        txt += f"- Dimensions de la grille : {self.height} x {self.width}\n"
        txt += "- Voisinages :\n"
        txt += str(self.neighbors)+"\n"
        valid = True
        for c1 in {(i, j) for i in range(self.height) for j in range(self.width)}:
            for c2 in self.neighbors[c1]:
                if c1 not in self.neighbors[c2]:
                    valid = False
                    break
            else:
                continue
            break
        txt += "- Structure cohérente\n" if valid else f"- Structure incohérente : {c1} X {c2}\n"
        return txt

    def __str__(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Représentation textuelle d'un objet Maze (en utilisant des caractères ascii)
        Retour:
             chaîne (str) : chaîne de caractères représentant le labyrinthe
        """
        txt = ""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += "   ┃" if (0,j+1) not in self.neighbors[(0,j)] else "    "
        txt += "   ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += "   ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else "    "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"

        return txt
    
    def add_wall(self, c1, c2):
        # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
            0 <= c1[1] < self.width and \
            0 <= c2[0] < self.height and \
            0 <= c2[1] < self.width, \
            f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        # Ajout du mur
        if c2 in self.neighbors[c1]:      # Si c2 est dans les voisines de c1
            self.neighbors[c1].remove(c2) # on le retire
        if c1 in self.neighbors[c2]:      # Si c3 est dans les voisines de c2
            self.neighbors[c2].remove(c1) # on le retire
            
    def remove_wall(self, c1, c2):
        # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
            0 <= c1[1] < self.width and \
            0 <= c2[0] < self.height and \
            0 <= c2[1] < self.width, \
            f"Erreur lors de la suppression d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        # Suppression du mur
        if c2 not in self.neighbors[c1]:  # Si c2 n'est pas dans les voisines de c1
            self.neighbors[c1].add(c2)    # on l'ajoute
        if c1 not in self.neighbors[c2]:  # Si c3 n'est pas dans les voisines de c2
            self.neighbors[c2].add(c1)    # on l'ajoute
    
    def fill(self):
        """Ajoute tous les murs possibles dans le labyrinthe"""
        for i in range(self.height):
            for j in range(self.width):
                if i > 0:
                    self.neighbors[(i,j)].remove((i-1,j))
                    self.neighbors[(i-1,j)].remove((i,j))
                if j > 0:
                    self.neighbors[(i,j)].remove((i,j-1))
                    self.neighbors[(i,j-1)].remove((i,j))
    
    def empty(self):
        for i in range(self.height):
            for j in range(self.width):
                if i > 0:
                    self.neighbors[(i,j)].add((i-1,j))
                    self.neighbors[(i-1,j)].add((i,j))
                if j > 0:
                    self.neighbors[(i,j)].add((i,j-1))
                    self.neighbors[(i,j-1)].add((i,j))
                    
    def get_walls(self):
        """
        Méthode renvoyant la liste des murs sous forme de tuples présents dans le labyrinthe
        """
        walls = []
        for i in range(self.height):
            for j in range(self.width):
                if i > 0 and (i-1,j) not in self.neighbors[(i,j)]:
                    walls.append(((i,j),(i-1,j)))
                if j > 0 and (i,j-1) not in self.neighbors[(i,j)]:
                    walls.append(((i,j),(i,j-1)))
        return walls
    
    def get_contiguous_cells(self, c):
        """
        Retourne la liste des cellules contigües à c dans la grille (sans s’occuper des éventuels murs)
        """
        i = c[0]
        j = c[1]
        result = []
        if i > 0:
            result.append((i-1, j))
        if i < self.height-1:
            result.append((i+1, j))
        if j > 0:
            result.append((i, j-1))
        if j < self.width-1:
            result.append((i, j+1))
        return result
    
    def get_reachable_cells(self, c):
        """
        Retourne la liste des cellules accessibles depuis c (c’est-à-dire les cellules contiguës à c qui sont dans le voisinage de c)
        """
        liste = []
        for i in self.neighbors[c]:
            liste.append(i)
        return liste
    
    @classmethod
    def gen_btree(cls,h,w):          
        maze = cls(h,w,False)
        for i in range (0,h-1):
            for j in range(0,w):
                if j < w -1:
                    c = random.choice([(i+1,j),(i,j+1)])
                    maze.remove_wall((i,j),c)
                else:
                    maze.remove_wall((i,j),(i+1,j))
        for i in range(0,w-1):
            maze.remove_wall((h-1,i),(h-1,i+1))
        return maze
                

    @classmethod
    def gen_sidewinder(cls, h, w):        
        maze = cls(h, w)
        for i in range(h):
            parcours = 0
            for j in range(w):
                if i > 0 and (j == w-1 or random.random() < 0.5): #Ici, random.random() < 0.5 correspond à la séléction du mur est ou sud.
                    c = random.choice(range(parcours, j+1))
                    maze.remove_wall((i-1, c), (i, c))
                    parcours = j+1
                elif j < w-1:
                    maze.remove_wall((i, j), (i, j+1))
        for i in range(0,w-1):
            maze.remove_wall((h-1,i),(h-1,i+1))
        return maze
    
    @classmethod
    def gen_fusion(cls, h, w):
        maze = cls(h, w)
        label = 0
        dictionnaire = {}
        walls = maze.get_walls()
        for i in range(len(walls)):
            dictionnaire[(walls[i][0])] = label
            label += 1
            dictionnaire[(walls[i][1])] = label
            label += 1
        random.shuffle(walls)
        changement_precedent = 0
        changement = 1
        while changement != changement_precedent:
            changement_precedent = changement # On aurait pus faire l'inverse, mais cela donne la possibilité de récupérer le nombre de changement au besoin
            for i in range(len(walls)):
                if dictionnaire.get(walls[i][0]) != dictionnaire.get(walls[i][1]):
                    cellule1 = walls[i][0]
                    cellule2 = walls[i][1]
                    maze.remove_wall(walls[i][0], walls[i][1])
                    label_a_garder = dictionnaire.get(walls[i][0])
                    label = dictionnaire.get(walls[i][1])
                    for cellule, label_present in dictionnaire.items():
                        if label_present == label:
                            dictionnaire[cellule] = label_a_garder
                    changement += 1
        return maze
                    
        
    @classmethod
    def gen_exploration(cls, h, w):
        maze = cls(h, w)
        visite = set()
        pile = []
        c = (random.randint(0, h-1), random.randint(0, w-1))
        visite.add(c)
        pile.append(c)
        while pile:
            c1 = pile.pop()
            tout_voisins_visite = True
            voisins_contig_c1 = maze.get_contiguous_cells(c1)
            voisins_non_visite = set(voisins_contig_c1) - visite
            for neighbor in voisins_non_visite:
                if neighbor not in visite:
                    tout_voisins_visite = False
            if not tout_voisins_visite:
                pile.append(c1)
                c2 = random.choice(list(voisins_non_visite))
                maze.remove_wall(c1, c2)
                visite.add(c2)
                pile.append(c2)
        return maze
        
    def overlay(self, content=None):
        """
        Rendu en mode texte, sur la sortie standard, \
        d'un labyrinthe avec du contenu dans les cellules
        Argument:
            content (dict) : dictionnaire tq content[cell] contient le caractère à afficher au milieu de la cellule
        Retour:
            string
        """
        if content is None:
            content = {(i,j):' ' for i in range(self.height) for j in range(self.width)}
        else:
            # Python >=3.9
            #content = content | {(i, j): ' ' for i in range(
            #    self.height) for j in range(self.width) if (i,j) not in content}
            # Python <3.9
            new_content = {(i, j): ' ' for i in range(self.height) for j in range(self.width) if (i,j) not in content}
            content = {**content, **new_content}
        txt = r""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += " "+content[(0,j)]+" ┃" if (0,j+1) not in self.neighbors[(0,j)] else " "+content[(0,j)]+"  "
        txt += " "+content[(0,self.width-1)]+" ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += " "+content[(i+1,j)]+" ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else " "+content[(i+1,j)]+"  "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"
        return txt

    def solve_dfs(self, start, stop):
        pile = [start]
        pred = {start: None}
        while pile:
            c = pile.pop()
            if c == stop:
                break
            for neighbor in self.get_reachable_cells(c):
                if neighbor not in pred:
                    pile.append(neighbor)
                    pred[neighbor] = c
        chemin = [stop]
        while chemin[-1] != start:
            chemin.append(pred[chemin[-1]])
        chemin.reverse()
        return chemin
    
    def solve_bfs(self, start, stop):
        file = [start]
        pred = {start: None}
        while file:
            c = file.pop(0)
            if c == stop:
                break
            for neighbor in self.get_reachable_cells(c):
                if neighbor not in pred:
                    file.append(neighbor)
                    pred[neighbor] = c
        chemin = [stop]
        while chemin[-1] != start:
            chemin.append(pred[chemin[-1]])
        chemin.reverse()
        return chemin

In [811]:
laby = Maze(4, 4)
print(laby.info())
print(laby)

**Informations sur le labyrinthe**
- Dimensions de la grille : 4 x 4
- Voisinages :
{(0, 0): set(), (0, 1): set(), (0, 2): set(), (0, 3): set(), (1, 0): set(), (1, 1): set(), (1, 2): set(), (1, 3): set(), (2, 0): set(), (2, 1): set(), (2, 2): set(), (2, 3): set(), (3, 0): set(), (3, 1): set(), (3, 2): set(), (3, 3): set()}
- Structure cohérente

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [812]:
laby.neighbors = {
    (0, 0): {(1, 0)},
    (0, 1): {(0, 2), (1, 1)},
    (0, 2): {(0, 1), (0, 3)},
    (0, 3): {(0, 2), (1, 3)},
    (1, 0): {(2, 0), (0, 0)},
    (1, 1): {(0, 1), (1, 2)},
    (1, 2): {(1, 1), (2, 2)},
    (1, 3): {(2, 3), (0, 3)},
    (2, 0): {(1, 0), (2, 1), (3, 0)},
    (2, 1): {(2, 0), (2, 2)},
    (2, 2): {(1, 2), (2, 1)},
    (2, 3): {(3, 3), (1, 3)},
    (3, 0): {(3, 1), (2, 0)},
    (3, 1): {(3, 2), (3, 0)},
    (3, 2): {(3, 1)},
    (3, 3): {(2, 3)}
}

print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋   ┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [813]:
laby.neighbors[(1,3)].remove((2,3))
laby.neighbors[(2,3)].remove((1,3))
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋━━━┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [814]:
laby.neighbors[(1, 3)].add((2, 3))
laby.neighbors[(2, 3)].add((1, 3))
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋   ┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [815]:
laby.neighbors[(1, 3)].remove((2, 3))
print(laby)
print(laby.info())

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃           ┃
┣   ╋   ╋━━━╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋━━━┫
┃           ┃   ┃
┣   ╋━━━╋━━━╋   ┫
┃           ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛

**Informations sur le labyrinthe**
- Dimensions de la grille : 4 x 4
- Voisinages :
{(0, 0): {(1, 0)}, (0, 1): {(1, 1), (0, 2)}, (0, 2): {(0, 1), (0, 3)}, (0, 3): {(0, 2), (1, 3)}, (1, 0): {(2, 0), (0, 0)}, (1, 1): {(0, 1), (1, 2)}, (1, 2): {(1, 1), (2, 2)}, (1, 3): {(0, 3)}, (2, 0): {(1, 0), (2, 1), (3, 0)}, (2, 1): {(2, 0), (2, 2)}, (2, 2): {(1, 2), (2, 1)}, (2, 3): {(3, 3), (1, 3)}, (3, 0): {(3, 1), (2, 0)}, (3, 1): {(3, 2), (3, 0)}, (3, 2): {(3, 1)}, (3, 3): {(2, 3)}}
- Structure incohérente : (2, 3) X (1, 3)



In [816]:
laby.neighbors[(2, 3)].remove((1,3))
c1 = (1, 3)
c2 = (2, 3)
if c1 in laby.neighbors[c2] and c2 in laby.neighbors[c1]:
    print(f"Il n'y a pas de mur entre {c1} et {c2} car elles sont mutuellement voisines")
elif c1 not in laby.neighbors[c2] and c2 not in laby.neighbors[c1]:
    print(f"Il y a un mur entre {c1} et {c2} car {c1} n'est pas dans le voisinage de {c2} et {c2} n'est pas dans le voisinage de {c1}")
else:
    print(f"Il y a une incohérence de réciprocité des voisinages de {c1} et {c2}")

Il y a un mur entre (1, 3) et (2, 3) car (1, 3) n'est pas dans le voisinage de (2, 3) et (2, 3) n'est pas dans le voisinage de (1, 3)


In [817]:
c1 = (1, 3)
c2 = (2, 3)
if c1 in laby.neighbors[c2] and c2 in laby.neighbors[c1]:
    print(f"{c1} est accessible depuis {c2} et vice-versa")
elif c1 not in laby.neighbors[c2] and c2 not in laby.neighbors[c1]:
    print(f"{c1} n'est pas accessible depuis {c2} et vice-versa")
else:
    print(f"Il y a une incohérence de réciprocité des voisinages de {c1} et {c2}")

(1, 3) n'est pas accessible depuis (2, 3) et vice-versa


In [818]:
L = []
for i in range(laby.height):
    for j in range(laby.width):
        L.append((i,j))
print(f"Liste des cellules : \n{L}")

Liste des cellules : 
[(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3)]


### **À faire**

Modifier le constructeur par défaut en lui ajoutant l’argument `empty`, un booléen qui indique si le graphe doit être créé avec aucun mur, ou avec tous les murs.

Modifier le corps de la méthode de telle manière que :

- si `empty` vaut `True`, chaque cellule a pour voisines celles qui lui sont contigües dans la grille ;
- si `empty` vaut `False`, aucune cellule n’a de voisines.

In [819]:
laby = Maze(4, 4, empty = True)
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛



In [820]:
laby = Maze(4, 4, empty = False)
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



## 4 Manipulation de labyrinthes

Nous allons avoir besoin de méthodes d’instance essentielles, pour construire et résoudre ces problèmes de labyrinthes.

Construisons d’abord, à titre d’exemple, la méthode `add_wall(c1, c2)` qui ajoute un mur entre entre la cellule `c1` et la cellule `c2`.

Ajouter un mur entre deux cellules revient à couper la possibilité de se déplacer de l’une à l’autre et inversement. Il s’agit donc de retirer `c1` des voisines de `c2`, et de retirer `c2` des voisines de `c1`.

Ce qui donne (version robuste) :

In [821]:
def add_wall(self, c1, c2):
    # Facultatif : on teste si les sommets sont bien dans le labyrinthe
    assert 0 <= c1[0] < self.height and \
        0 <= c1[1] < self.width and \
        0 <= c2[0] < self.height and \
        0 <= c2[1] < self.width, \
        f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
    # Ajout du mur
    if c2 in self.neighbors[c1]:      # Si c2 est dans les voisines de c1
        self.neighbors[c1].remove(c2) # on le retire
    if c1 in self.neighbors[c2]:      # Si c3 est dans les voisines de c2
        self.neighbors[c2].remove(c1) # on le retire

Exemple d’utilisation :

In [822]:
laby = Maze(5, 5, empty = True)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [823]:
laby.add_wall((0,0), (0,1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃               ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



### **À faire**

Écrire les méthodes d’instance suivantes :

- `remove_wall(c1, c2)` qui supprime un mur entre deux cellules
- `get_walls()` qui retourne la liste de **tous les murs** sous la forme d’une liste de `tuple` de cellules
- `fill()` qui ajoute tous les murs possibles dans le labyrinthe
- `empty()` qui supprime tous les murs du labyrinthe
- `get_contiguous_cells(c)` qui retourne la liste des cellules contigües à `c` **dans la grille** (sans s’occuper des éventuels murs)
- `get_reachable_cells(c)` qui retourne la liste des cellules accessibles depuis `c` (c’est-à-dire les cellules contiguës à c qui sont dans le voisinage de c)

In [824]:
laby = Maze(5, 5, empty = True)
laby.fill()
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [825]:
laby.remove_wall((0, 0), (0, 1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃       ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [826]:
laby.empty()
laby.add_wall((0, 0), (0, 1))
laby.add_wall((0, 1), (1, 1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃               ┃
┣   ╋━━━╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [827]:
laby = Maze(4, 4, empty = False)
print(laby.get_walls())
print(laby)

[((0, 1), (0, 0)), ((0, 2), (0, 1)), ((0, 3), (0, 2)), ((1, 0), (0, 0)), ((1, 1), (0, 1)), ((1, 1), (1, 0)), ((1, 2), (0, 2)), ((1, 2), (1, 1)), ((1, 3), (0, 3)), ((1, 3), (1, 2)), ((2, 0), (1, 0)), ((2, 1), (1, 1)), ((2, 1), (2, 0)), ((2, 2), (1, 2)), ((2, 2), (2, 1)), ((2, 3), (1, 3)), ((2, 3), (2, 2)), ((3, 0), (2, 0)), ((3, 1), (2, 1)), ((3, 1), (3, 0)), ((3, 2), (2, 2)), ((3, 2), (3, 1)), ((3, 3), (2, 3)), ((3, 3), (3, 2))]
┏━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛



In [828]:
laby.get_contiguous_cells((2,2))

[(1, 2), (3, 2), (2, 1), (2, 3)]

In [829]:
laby = Maze(5, 5, False)
laby.remove_wall((1,2),(2,2))
laby.remove_wall((3,2),(2,2))
laby.get_reachable_cells((2,2))

[(3, 2), (1, 2)]

## 5 Génération

___

Nous allons maintenant nous intéresser aux algorithmes permettant de générer des labyrinthes parfaits.

Nous allons commencer par implémenter deux classiques assez simples, l’un reposant sur les arbres binaires et le second, appelé sidewinder. Nous verrons aussi deux algorithmes un peu plus avancés, qu’on retrouve notamment dans l’article wikipedia consacré à la modélisation des labytrinthes :

- l’algorithme de génération par **fusion de chemins** (qui peut-être vu comme une forme de l’algorithme de Kruskal, qui permet de détermnier un **arbre couvrant de poids minimal** dans un graphe non-orienté valué)
- l’algorithme de génération par **exploration exhaustive** (qui utilise un parcours de graphe, en profondeur ou en largeur)

On terminera par l’algorithme de Wilson.

### 5.1 Arbre binaire

___

L’algorithme de génération par arbre binaire consiste à générer… un arbre binaire comme support du labyrinthe.

In [830]:
laby = Maze.gen_btree(7,7)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃           ┃       ┃   ┃   ┃
┣━━━╋━━━╋   ╋━━━╋   ╋   ╋   ┫
┃   ┃       ┃   ┃   ┃       ┃
┣   ╋━━━╋   ╋   ╋   ╋━━━╋   ┫
┃                   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ┫
┃   ┃   ┃   ┃       ┃       ┃
┣   ╋   ╋   ╋━━━╋   ╋━━━╋   ┫
┃       ┃   ┃   ┃           ┃
┣━━━╋   ╋   ╋   ╋━━━╋━━━╋   ┫
┃           ┃       ┃       ┃
┣━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ┫
┃                           ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┛



### 5.2 Sidewinder

___

L’algorithme de génération *sidewinder* ressemble beaucoup au précédent. L’idée est de procéder ligne par ligne, d’OUEST en EST, en choisissant aléatoirement de casser le mur EST d’une cellule. Pour chaque séquence de cellules voisines (connectées) créée sur la ligne, on casse un mur SUD au hasard d’une de ces cellules (une séquence peut être constituée d’une seule cellule).

In [831]:
laby = Maze.gen_sidewinder(5, 5)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃                   ┃
┣   ╋━━━╋   ╋   ╋   ┫
┃       ┃   ┃   ┃   ┃
┣   ╋━━━╋   ╋━━━╋   ┫
┃       ┃   ┃       ┃
┣━━━╋   ╋   ╋━━━╋   ┫
┃       ┃       ┃   ┃
┣   ╋━━━╋━━━╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



### 5.3 Fusion de chemins

___

L’algorithme de fusion de chemins consiste à partir d’un labyrinthe « plein », puis à casser des murs au hasard en évitant de créer des cycles. Puisqu’un labyrinthe parfait est un arbre, et qu’un arbre à sommets a exactement n-1 arêtes, il suffira d’abattre n-1 murs (soit h x w -1 si h et w désignent respectivement le nombre de lignes et le nombre de colonnes).

Pour éviter de créer des cycles, on utilise un mécanisme de labélisation des cellules (avec des entiers). Lorsqu’on casse un mur depuis une cellule, le label de la cellule « se propage » dans la zone découverte. Mais on n’ouvrira un mur que lorsque le label de la cellule courante est différent du label de la cellule qui est de l’autre côté du mur.

In [840]:
laby = Maze.gen_fusion(10,10)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃       ┃       ┃   ┃               ┃   ┃
┣━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ┫
┃                                       ┃
┣   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃       ┃   ┃                           ┃
┣━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━┫
┃   ┃   ┃       ┃       ┃               ┃
┣   ╋━━━╋   ╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━┫
┃       ┃   ┃   ┃   ┃   ┃       ┃       ┃
┣━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━┫
┃           ┃               ┃       ┃   ┃
┣━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋━━━╋   ╋   ┫
┃               ┃   ┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ┫
┃       ┃       ┃   ┃                   ┃
┣   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ┫
┃   ┃   ┃       ┃   ┃       ┃   ┃       ┃
┣   ╋   ╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ┫
┃   ┃   ┃                   ┃       ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┛



### 5.4 Exploration exhaustive

___

Une deuxième idée consiste à « explorer » aléatoirement le labyrinthe, à la manière d’un parcours en profondeur, en cassant les murs à mesure qu’on avance

In [833]:
laby = Maze.gen_exploration(15,15)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃               ┃           ┃                   ┃           ┃
┣   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ┫
┃       ┃       ┃   ┃   ┃   ┃           ┃   ┃   ┃       ┃   ┃
┣   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━┫
┃   ┃       ┃   ┃       ┃       ┃   ┃           ┃   ┃       ┃
┣━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ┫
┃           ┃       ┃       ┃       ┃   ┃           ┃   ┃   ┃
┣   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ┫
┃       ┃                   ┃   ┃       ┃   ┃   ┃           ┃
┣   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ┫
┃   ┃           ┃       ┃   ┃               ┃       ┃       ┃
┣   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋━━━┫
┃   ┃   ┃       ┃   ┃       ┃                       ┃       ┃
┣   ╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋   ┫
┃       ┃       ┃           ┃   ┃           ┃           ┃   ┃
┣━━━╋   

### 5.5 L’algorithme de Wilson

___

Terminons avec un algorithme plus amusant, qui donne des labyrinthes très intéressants : l’algorithme de Wilson. Il repose sur les **marches aléatoires**.

L’idée est la suivante : on va construire le labyrinthe en essayant des chemins aléatoires, jusqu’à obtention d’une arborescence…

In [834]:
#code

## 6 Résolution

In [835]:
laby = Maze(4,4, empty = True)
print(laby.overlay({
    (0, 0):'c',
    (0, 1):'o',
    (1, 1):'u',
    (2, 1):'c',
    (2, 2):'o',
    (3, 2):'u',
    (3, 3):'!'}))

┏━━━┳━━━┳━━━┳━━━┓
┃ c   o         ┃
┣   ╋   ╋   ╋   ┫
┃     u         ┃
┣   ╋   ╋   ╋   ┫
┃     c   o     ┃
┣   ╋   ╋   ╋   ┫
┃         u   ! ┃
┗━━━┻━━━┻━━━┻━━━┛



In [836]:
laby = Maze(4,4, empty = True)
path = {(0, 0): '@',
        (1, 0): '*',
        (1, 1): '*',
        (2, 1): '*',
        (2, 2): '*',
        (3, 2): '*',
        (3, 3): '§'}
print(laby.overlay(path))

┏━━━┳━━━┳━━━┳━━━┓
┃ @             ┃
┣   ╋   ╋   ╋   ┫
┃ *   *         ┃
┣   ╋   ╋   ╋   ┫
┃     *   *     ┃
┣   ╋   ╋   ╋   ┫
┃         *   § ┃
┗━━━┻━━━┻━━━┻━━━┛



### 6.1 Résolution par parcours

___ 

L’algorithme le plus évident pour résoudre un problème de labyrinthe, consiste à adapter le parcours « en profondeur d’abord » de l’arborescence associée au labyrinthe

In [837]:
laby = Maze.gen_exploration(15, 15)
solution = laby.solve_dfs((0, 0), (14, 14))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(14, 14)] = 'A'
print(laby.overlay(str_solution))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ D   *   *   * ┃                               ┃           ┃
┣   ╋━━━╋   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━┫
┃   ┃       ┃ *   * ┃       ┃       ┃           ┃           ┃
┣━━━╋   ╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ┫
┃       ┃       ┃ * ┃           ┃   ┃                       ┃
┣   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ┫
┃   ┃       ┃     *   *   *   * ┃   ┃                   ┃   ┃
┣   ╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ┫
┃   ┃                   ┃   ┃ * ┃   ┃   ┃           ┃       ┃
┣   ╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━┫
┃       ┃       ┃   ┃   ┃ *   * ┃           ┃       ┃   ┃   ┃
┣━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ┫
┃           ┃       ┃   ┃ * ┃               ┃       ┃       ┃
┣   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━┫
┃   ┃       ┃       ┃   ┃ * ┃   ┃ *   * ┃       ┃           ┃
┣   ╋━━━

In [838]:
solution = laby.solve_bfs((0, 0), (14, 14))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(14, 14)] = 'A'
print(laby.overlay(str_solution))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ D   *   *   * ┃                               ┃           ┃
┣   ╋━━━╋   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━┫
┃   ┃       ┃ *   * ┃       ┃       ┃           ┃           ┃
┣━━━╋   ╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ┫
┃       ┃       ┃ * ┃           ┃   ┃                       ┃
┣   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ┫
┃   ┃       ┃     *   *   *   * ┃   ┃                   ┃   ┃
┣   ╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ┫
┃   ┃                   ┃   ┃ * ┃   ┃   ┃           ┃       ┃
┣   ╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━┫
┃       ┃       ┃   ┃   ┃ *   * ┃           ┃       ┃   ┃   ┃
┣━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ┫
┃           ┃       ┃   ┃ * ┃               ┃       ┃       ┃
┣   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━┫
┃   ┃       ┃       ┃   ┃ * ┃   ┃ *   * ┃       ┃           ┃
┣   ╋━━━